In [3]:

import json
import pickle
import tensorflow
import keras
import random
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout
from keras.layers.embeddings import Embedding
import numpy as np
import pandas as pd
from tkinter import *


Using TensorFlow backend.


In [11]:
classes = []
data_file = open('chatbot_intent.json',encoding="utf-8").read()
intents = json.loads(data_file,strict = False)

In [5]:
data  = pd.read_csv("data_1.csv")
X = data.drop('infectionProb',axis = 1)
Y = data['infectionProb']
from sklearn.ensemble import RandomForestClassifier
model_class = RandomForestClassifier()
model_class.fit(X,Y)

C:\Users\dell\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [12]:
tokenizer_obj =Tokenizer()
pattern_list = []
pattern_list_2 = []
for intent in intents['intents']:
    for pattern in intent['patterns']:
        pattern_list.append(pattern.lower())
        pattern_list_2.append(intent['tag'])
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [14]:
tokenizer_obj = Tokenizer()
tokenizer_obj.fit_on_texts(pattern_list)

In [13]:
max_length = max([len(s.split()) for s in pattern_list])

In [15]:
X_train = tokenizer_obj.texts_to_sequences(pattern_list)

In [16]:
X_train_pad = pad_sequences(X_train,maxlen=max_length,padding='post')

In [17]:
def one_hot_vector(expected):
    res = [0] * len(classes)
    for i,w in enumerate(classes):
        if(w == expected):
            res[i] = 1
    return res    

In [18]:
y = []
for i in pattern_list_2:
    y.append(one_hot_vector(i))

In [19]:
y = np.array(y)
X_train_pad = np.array(X_train_pad)

In [20]:
vocab_size = len(tokenizer_obj.word_index) + 1
Embedding_dim = 60

In [4]:
from keras.models import load_model
model = load_model('chatbot_model_final.h5')
classes = pickle.load(open('classes.pkl','rb'))
intents = json.loads(open('chatbot_intent.json',encoding="utf-8").read(),strict=False)


C:\Users\dell\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [6]:
def clean_sentence(sentence,max_length,model):
    a = []
    a.append(sentence.lower())
    a = tokenizer_obj.texts_to_sequences(a)
    a = pad_sequences(a,maxlen=max_length,padding='post')
    temp = a[0].reshape(-1,max_length)
    res = model.predict_classes(temp)
    return res

def predict_all(sentence,max_length,model):
    res = clean_sentence(sentence,max_length,model)
    return res[0]      

def getResponse(res,intents,classes):
    for intent in intents['intents']:
        if(intent['tag'] == classes[res]):
            k = random.choice(intent['responses'],)
    return k    


def chatbot_response(msg,model,max_length,intents,classes):
    ints = predict_all(msg,max_length,model)
    res = getResponse(ints,intents,classes)
    return res

In [8]:
def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)
    
    if msg != '':
        ChatLog.config(state = NORMAL)
        ChatLog.insert(END,"You: " + msg + '\n\n')
        ChatLog.config(foreground = "black" , font = ("Arial,12"))
        
        res = chatbot_response(msg.lower(),model,max_length,intents,classes)
        
        ChatLog.insert(END,"Bot: " + res +'\n\n')
        
        ChatLog.config(state = DISABLED)
        ChatLog.yview(END)    

In [7]:
store = []

def myclick():
    top = Toplevel(base)
    top.title("Symptom_checker")
    
    temp_var = StringVar()
    body_pain_var = StringVar()
    age_var = StringVar()
    nose_var = StringVar()
    breath_var = StringVar()
    
    widget1 = Label(top,text = "Enter your body temperature")
    e1 = Entry(top)
    widget1.grid(row = 0,column = 0)
    e1.grid(row = 0,column = 1)
    
    widget2 = Label(top,text = "Do you have body pain (Enter 1 or 0)")
    e2 = Entry(top)
    widget2.grid(row = 1,column = 0)
    e2.grid(row = 1,column = 1)
    
    widget3 = Label(top,text = "Enter your age (in digits)")
    e3 = Entry(top)
    widget3.grid(row = 2,column = 0)
    e3.grid(row = 2,column = 1)
    
    widget4 = Label(top,text = "Do you have runnyNose (Enter 1 or 0)")
    e4 = Entry(top)
    widget4.grid(row = 3,column = 0)
    e4.grid(row = 3,column = 1)
    
    widget5 = Label(top,text = "Do you have difficulty in breathing (Enter 1 or 0)")
    e5 = Entry(top)
    widget5.grid(row = 4,column = 0)
    e5.grid(row = 4,column = 1)
    
    widget6 = Label(top,text="*The results are experimental*")
    widget6.grid(row = 6,column = 1)
    
    def printresult():
        
        temp = np.array(store)
        temp = temp.reshape(-1,5)
        temp = pd.DataFrame(temp)
        z = model_class.predict(temp)
        if(z[0] == 1):
            widget7 = Label(top,text = "You have a high probabilty of corona infection")
            widget7.grid(row = 7,column = 1)
        
        if(z[0] == 0):
            widget7 = Label(top,text = "You have a low probabilty of corona infection")
            widget7.grid(row = 7,column = 1)
            
        store.clear()    
    
    def calculate():
        
        temp_var = e1.get()
        body_pain_var = e2.get()
        age_var = e3.get()
        nose_var = e4.get()
        breath_var = e5.get()
        
        
        store.append(float(temp_var))
        store.append(int(body_pain_var,10))
        store.append(int(age_var,10))
        store.append(int(nose_var,10))
        store.append(int(breath_var,10))
        
        
    
        e1.delete(0,END)
        e2.delete(0,END)
        e3.delete(0,END)
        e4.delete(0,END)
        e5.delete(0,END)
        
        printresult()
        
        
    
    submit_button = Button(top,text="Submit and check",command = calculate)
    submit_button.grid(row=5,column = 1)    

In [21]:
base = Tk()
base.title("Covid-Assistant")
base.geometry("400x500")
base.resizable(width = FALSE,height=FALSE)

ChatLog = Text(base,bd = 0,bg="#EBE3FE",height="8",width="150",font="Arial")
scrollbar = Scrollbar(base,command = ChatLog.yview,cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

SendButton = Button(base,font = ("Veradana",15,'bold'),text="Send",width ="10",height="5",bd=0,bg="#2A789C",activebackground="#3c9d9b",fg='#ffffff',
                    command= send)

EntryBox = Text(base,bd=0,bg="#D5FCD3",width="29", height="5", font="Arial")
myButton = Button(base,text="Symptom Checker",command = myclick,font = "bold")



scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)
myButton.place(x = 235,y=360)


base.mainloop()



In [ ]:
##Implemet till here for only if you want to use chatbot after this is the architecture of the network used

In [58]:
model = Sequential()
model.add(Embedding(vocab_size,Embedding_dim,input_length=max_length))
model.add(LSTM(units = 128,return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(units = 128))
model.add(Dropout(0.5))
model.add(Dense(len(classes),activation='softmax'))

In [59]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [60]:
hist = model.fit(X_train_pad,y,batch_size = 10,epochs = 100,verbose=2)

C:\Users\dell\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
 - 1s - loss: 0.1426 - accuracy: 0.9677
Epoch 2/100
 - 0s - loss: 0.1417 - accuracy: 0.9677
Epoch 3/100
 - 0s - loss: 0.1409 - accuracy: 0.9677
Epoch 4/100
 - 0s - loss: 0.1391 - accuracy: 0.9677
Epoch 5/100
 - 0s - loss: 0.1366 - accuracy: 0.9677
Epoch 6/100
 - 0s - loss: 0.1318 - accuracy: 0.9677
Epoch 7/100
 - 0s - loss: 0.1273 - accuracy: 0.9680
Epoch 8/100
 - 0s - loss: 0.1257 - accuracy: 0.9677
Epoch 9/100
 - 0s - loss: 0.1230 - accuracy: 0.9680
Epoch 10/100
 - 0s - loss: 0.1200 - accuracy: 0.9675
Epoch 11/100
 - 0s - loss: 0.1178 - accuracy: 0.9691
Epoch 12/100
 - 0s - loss: 0.1154 - accuracy: 0.9687
Epoch 13/100
 - 0s - loss: 0.1100 - accuracy: 0.9694
Epoch 14/100
 - 0s - loss: 0.1086 - accuracy: 0.9694
Epoch 15/100
 - 0s - loss: 0.1054 - accuracy: 0.9698
Epoch 16/100
 - 0s - loss: 0.1100 - accuracy: 0.9698
Epoch 17/100
 - 0s - loss: 0.1026 - accuracy: 0.9703
Epoch 18/100
 - 0s - loss: 0.0992 - accuracy: 0.9696
Epoch 19/100
 - 0s - loss: 0.0972 - accuracy: 0.9696
Ep

In [61]:
model.save('chatbot_model_final.h5',hist)

In [62]:
classes = pickle.dump(classes,open('classes.pkl','wb'))
X_train = pickle.dump(pattern_list,open('pattern_list.pkl','wb'))
y_train = pickle.dump(y,open('y.pkl','wb'))

In [64]:
from tkinter import *
